In [ ]:
# -*- coding: utf-8 -*-
# config
ImportFileDirectory = r''
SheetName = r''
StartCellColumn = r'' # eg. A
StartCellRow_N = r'' # eg. 1
EndCellColumn = r'' # eg. Z
EndCellRow_N = r'' # eg. 99
ExportFilePath = r''

In [ ]:
# definitions
import xlrd, xlsxwriter
from os import listdir
from multiprocessing import Process, Queue, freeze_support

# define variables
CellColumnPool, finals = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', []

def Selection():
    global finals
    FileNameList = listdir(ImportFileDirectory)
    FileNameList = [f for f in FileNameList if f[-4:] == 'xlsx' and f[:2] != '._']
    ProcessList = []
    q = Queue()
    if StartCellRow_N + 1 == EndCellRow_N:
        if StartCellColumn == EndCellColumn:
            for f in FileNameList:
                p = Process(target = Files_Row_Column, args = (q, f, ImportFileDirectory, SheetName, StartCellRow_N, StartCellColumn_N, ))
                ProcessList.append(p)
                p.start()
        else:
            for f in FileNameList:
                p = Process(target = Files_Row_Columns, args = (q, f, ImportFileDirectory, SheetName, StartCellRow_N, StartCellColumn_N, EndCellColumn_N, ))
                ProcessList.append(p)
                p.start()
    else:
        if StartCellColumn == EndCellColumn:
            for f in FileNameList:
                p = Process(target = Files_Rows_Column, args = (q, f, ImportFileDirectory, SheetName, StartCellRow_N, EndCellRow_N, StartCellColumn_N, ))
                ProcessList.append(p)
                p.start()
        else:
            for f in FileNameList:
                p = Process(target = Files_Rows_Columns, args = (q, f, ImportFileDirectory, SheetName, StartCellRow_N, EndCellRow_N, StartCellColumn_N, EndCellColumn_N, ))
                ProcessList.append(p)
                p.start()
    for p in ProcessList:
        finals.append(q.get())
    for p in ProcessList:
        p.join()
    if StartCellRow_N + 1 == EndCellRow_N:
        if StartCellColumn == EndCellColumn:
            ExportExcel(finals)
        else:
            ExportExcel(finals)
    else:
        if StartCellColumn == EndCellColumn:
            ExportExcel(finals)
        else:
            ExportExcels(finals)

def Files_Row_Column(q, f, ImportFileDirectory, SheetName, i, j):
    result = [f]
    ExcelFile = xlrd.open_workbook(ImportFileDirectory + f, encoding_override = 'utf-8')
    Sheet = ExcelFile.sheet_by_name(SheetName)
    result.append(Sheet.cell_value(i, j))
    q.put(result)

def Files_Row_Columns(q, f, ImportFileDirectory, SheetName, i, StartCellColumn_N, EndCellColumn_N):
    result = [f]
    ExcelFile = xlrd.open_workbook(ImportFileDirectory + f, encoding_override = 'utf-8')
    Sheet = ExcelFile.sheet_by_name(SheetName)
    for Column in range(StartCellColumn_N, EndCellColumn_N):
        result.append(Sheet.cell_value(i, Column))
    q.put(result)

def Files_Rows_Column(q, f, ImportFileDirectory, SheetName, StartCellRow_N, EndCellRow_N, j):
    result = [f]
    ExcelFile = xlrd.open_workbook(ImportFileDirectory + f, encoding_override = 'utf-8')
    Sheet = ExcelFile.sheet_by_name(SheetName)
    for Row in range(StartCellRow_N, EndCellRow_N):
        result.append(Sheet.cell_value(Row, j))
    q.put(result)

def Files_Rows_Columns(q, f, ImportFileDirectory, SheetName, StartCellRow_N, EndCellRow_N, StartCellColumn_N, EndCellColumn_N):
    result = []
    ExcelFile = xlrd.open_workbook(ImportFileDirectory + f, encoding_override = 'utf-8')
    Sheet = ExcelFile.sheet_by_name(SheetName)
    for Row in range(StartCellRow_N, EndCellRow_N):
        row = [f]
        for Column in range(StartCellColumn_N, EndCellColumn_N):
            try:
                row.append(Sheet.cell_value(Row, Column))
            except:
                row.append('')
        result.append(row)
    q.put(result)

def ExportExcel(Breakdown):
    workbook = xlsxwriter.Workbook(ExportFilePath)
    worksheet = workbook.add_worksheet()
    for m in range(0, len(Breakdown)):
        for n in range(0, len(Breakdown[m])):
            worksheet.write(m, n, Breakdown[m][n])
    workbook.close()

def ExportExcels(Breakdown):
    workbook = xlsxwriter.Workbook(ExportFilePath)
    worksheet = workbook.add_worksheet()
    for l in range(0, len(Breakdown)):
        for m in range(0, len(Breakdown[l])):
            for n in range(0, len(Breakdown[l][m])):
                worksheet.write(l * len(Breakdown[l]) + m, n, Breakdown[l][m][n])
    workbook.close()

In [ ]:
# pre-process
if len(StartCellColumn) == 1:
    StartCellColumn_N = CellColumnPool.index(StartCellColumn)
elif len(StartCellColumn) == 2:
    StartCellColumn_N = (CellColumnPool.index(StartCellColumn[0]) + 1) * 26 + CellColumnPool.index(StartCellColumn[1])
else:
    print('Not Supported')
StartCellRow_N = int(StartCellRow_N) - 1
if len(EndCellColumn) == 1:
    EndCellColumn_N = CellColumnPool.index(EndCellColumn) + 1
elif len(EndCellColumn) == 2:
    EndCellColumn_N = (CellColumnPool.index(EndCellColumn[0]) + 1) * 26 + CellColumnPool.index(EndCellColumn[1]) + 1
else:
    print('Not Supported')
EndCellRow_N = int(EndCellRow_N)
# body
if __name__ == '__main__':
    freeze_support()
    Selection()